![]( https://docs.google.com/uc?export=download&id=1oKIi5t0GlSKAoPTIt1f-DcOu0GcYHDD4)

In [1]:
# FGA Big Data Using Python Kelas B 
# =====================================================================
# Kolaborasi Instruktur FILKOM x FMIPA UB 2021
# Di Bawah Naungan Kominfo Pusat x Cisco
# =====================================================================
# Instruktur: Imam Cholissodin, S.Si., M.Kom. 
# Asisten: Yonas Asmara

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/My Drive")
path = "Cisco Big Data Using Python 2021/FGA Big Data Using Python - Kelas B - utk Peserta/Sesi 1 - 25 Python"
if not os.path.exists(path):
    os.makedirs(path)
os.chdir("/content/drive/My Drive/Cisco Big Data Using Python 2021/FGA Big Data Using Python - Kelas B - utk Peserta/Sesi 1 - 25 Python")
if not os.path.exists("images"):
  !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-A9hoqmtpKWBAu_QjyqUxpcuBXkJ-Gk8' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-A9hoqmtpKWBAu_QjyqUxpcuBXkJ-Gk8" -O 'images.tar' && rm -rf /tmp/cookies.txt
  !tar -xv -C './' -f './images.tar'

!pwd
!ls -l -a --block-size=K

/content/drive/My Drive/Cisco Big Data Using Python 2021/FGA Big Data Using Python - Kelas B - utk Peserta/Sesi 1 - 25 Python
total 728K
-rw------- 1 root root   1K Jul 23 01:59 aritmatika.py
-rw------- 1 root root   1K Aug  6 02:02 Data_Generate_Reg.csv
drwx------ 2 root root   4K Aug 11 22:30 flask-sqlite
drwx------ 2 root root   4K Jul 20 09:59 images
-rw------- 1 root root 570K Jul 23 01:19 images.tar
drwx------ 2 root root   4K Jul 23 02:05 LATIHAN_PYTHON
drwx------ 2 root root   4K Jul 23 01:52 __pycache__
-rw------- 1 root root 141K Aug  9 02:16 PySpark_SOM_by_Imam_Cholissodin_On_Colab.ipynb


In [3]:
os.chdir("/content/drive/My Drive/Cisco Big Data Using Python 2021/FGA Big Data Using Python - Kelas B - utk Peserta/Sesi 1 - 25 Python/flask-sqlite/flask-sqlite-login")

In [ ]:
!pwd

/content/drive/My Drive/Cisco Big Data Using Python 2021/FGA Big Data Using Python - Kelas B - utk Peserta/Sesi 1 - 25 Python/flask-sqlite/flask-sqlite-login


In [4]:
!pip install flask-ngrok
!pip install flask==0.12.2  # Newer versions of flask don't work in Colab
                            # See https://github.com/plotly/dash/issues/257

# !pip install flask
!pip install SQLAlchemy
!pip install Flask-SQLAlchemy

     |████████████████████████████████| 83 kB 953 kB/s 
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4


In [7]:
# flask_ngrok_example.py
from flask import Flask,render_template,flash, redirect,url_for,session,logging,request
from flask_sqlalchemy import SQLAlchemy
from flask_ngrok import run_with_ngrok

# !pip install SQLAlchemy
# !pip install Flask-SQLAlchemy

# app = Flask(__name__)
# app = Flask(__name__, static_url_path='/static')
app = Flask(__name__, static_folder='static')
run_with_ngrok(app)  # Start ngrok when app is run

# app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:////fga_big_data.db'
db = SQLAlchemy(app)

class user(db.Model):
  # __tablename__ = 'user'
  id = db.Column(db.Integer, primary_key=True)
  # Mail 	Password 	Name 	Level
  Mail = db.Column(db.Text) # sbg Username
  Password = db.Column(db.Text)
  Name = db.Column(db.Text)
  Level = db.Column(db.Text)

  # username = Mail
  # password = Password

@app.route("/")
def index():
  return render_template("index.html")

@app.route("/login",methods=["GET", "POST"])
def login():
  msg = ""
  if request.method == "POST":
    mail = request.form["mail"]
    passw = request.form["passw"]
    login = user.query.filter_by(Mail=mail, Password=passw).first()
    print(login)
    if login is not None:
      return redirect(url_for("bigdataApps"))
    elif login is None:
      msg = "Masukkan Username (Email) dan Password dgn Benar"
      print(msg)
  return render_template("login.html", msg = msg)

@app.route("/register", methods=["GET", "POST"])
def register():
  if request.method == "POST":
    mail = request.form['mail']
    uname = request.form['uname']
    passw = request.form['passw']
    register = user(Mail = mail, Name = uname, Password = passw)
    db.session.add(register)
    db.session.commit()
    return redirect(url_for("login"))
    
  return render_template("register.html")

@app.route("/bigdataApps", methods=["GET", "POST"])
def bigdataApps():
  if request.method == 'POST':
    import pandas as pd
    import numpy as np
    dataset = request.FILES['inputDataset']
    # dataset = request.files['inputDataset']
    #dataset = 'dataset_dump.csv'

    persentase_data_training = 90
    banyak_fitur = int(request.POST['banyakFitur'])
    banyak_hidden_neuron = int(request.POST['banyakHiddenNeuron'])
    dataset = pd.read_csv(dataset, delimiter=';', names = ['Tanggal', 'Harga'], usecols=['Harga'])
    minimum = int(dataset.min()-10000)
    maksimum = int(dataset.max()+10000)
    new_banyak_fitur = banyak_fitur + 1
    hasil_fitur = []
    for i in range((len(dataset)-new_banyak_fitur)+1):
      kolom = []
      j = i
      while j < (i+new_banyak_fitur):
        kolom.append(dataset.values[j][0])
        j += 1
      hasil_fitur.append(kolom)
    hasil_fitur = np.array(hasil_fitur)
    data_normalisasi = (hasil_fitur - minimum)/(maksimum - minimum)
    data_training = data_normalisasi[:int(persentase_data_training*len(data_normalisasi)/100)]
    data_testing = data_normalisasi[int(persentase_data_training*len(data_normalisasi)/100):]
    
    #Training
    bobot = np.random.rand(banyak_hidden_neuron, banyak_fitur)
    bias = np.random.rand(banyak_hidden_neuron)
    h = 1/(1 + np.exp(-(np.dot(data_training[:, :banyak_fitur], np.transpose(bobot)) + bias)))
    h_plus = np.dot(np.linalg.inv(np.dot(np.transpose(h),h)),np.transpose(h))
    output_weight = np.dot(h_plus, data_training[:, banyak_fitur])
    
    #Testing
    h = 1/(1 + np.exp(-(np.dot(data_testing[:, :banyak_fitur], np.transpose(bobot)) + bias)))
    predict = np.dot(h, output_weight)
    predict = predict * (maksimum - minimum) + minimum
    
    #MAPE
    aktual = np.array(hasil_fitur[int(persentase_data_training*len(data_normalisasi)/100):, banyak_fitur])
    mape = np.sum(np.abs(((aktual - predict)/aktual)*100))/len(predict)
    return render_template('bigdataApps.html', {
        'y_aktual' : list(aktual),
        'y_prediksi' : list(predict),
        'mape' : mape
        })
  else:
    return render_template('bigdataApps.html')

@app.route("/logout")
def logout():
  session = None
  return redirect("/")

if __name__ == '__main__':
  db.create_all()
  app.run()
  # If address is in use, may need to terminate other sessions:
  # Runtime > Manage Sessions > Terminate Other Sessions  

/usr/local/lib/python3.7/dist-packages/flask_sqlalchemy/__init__.py:873: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://dee407af8867.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [15/Aug/2021 15:09:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:13] "GET /static/css/button2.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:15] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2021 15:09:17] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:17] "GET /static/css/main.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:17] "GET /static/css/font-awesome.min.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:17] "GET /static/css/util.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:17] "GET /static/css/material-design-iconic-font.min.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:17] "GET /static/css/font-awesome.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:17] "GET /static/css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:18] "GET /static/css/animate2.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:19] "GET /static/css/hamburgers.min.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Au

<user 1>


127.0.0.1 - - [15/Aug/2021 15:09:28] "GET /bigdataApps HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:29] "GET /static/css/chartist-plugin-tooltip.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:29] "GET /static/css/morris.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:29] "GET /static/css/chartist.min.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:29] "GET /static/css/sidebar-nav.min.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:29] "GET /static/css/jquery.toast.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:29] "GET /static/css/fullcalendar.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:29] "GET /static/css/animate.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:29] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:29] "GET /static/css/spinners.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:29] "GET /static/css/blue-dark.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2021 15:09:30] "GET /static/js/jquery.min.js HTT

In [ ]:
@app.route("/logout")
def logout():
  session = None
  return redirect("/")